In [1]:
# function calling을 사용하지 않으면 USA와 South Korea에 대한 답변 format이 달라진다.
# 첫 번째 응답은 주를 숫자(한 국가의 총 주 수)로 반환하고, 두 번째 응답은 주를 배열(주)로 반환

In [4]:
import os
import openai
import json
from openai import OpenAI
import keyring

client = OpenAI(
    api_key=keyring.get_password('openai', 'key_for_mac')
)

prompt = ''' 
인구, 수도, 주, 미국. JSON 객체로 반환
'''

completion = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{'role':'user', 'content':prompt}],
    max_tokens=1024,
    temperature=0
)
print(completion)
json.loads(completion.choices[0].message.content)

ChatCompletion(id='chatcmpl-9pTeJUN1zhtLqVmRRLhji7tKnRl34', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "인구": "331,002,651",\n  "수도": "Washington, D.C.",\n  "주": "50",\n  "국가": "미국"\n}', role='assistant', function_call=None, tool_calls=None))], created=1722055727, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=43, prompt_tokens=27, total_tokens=70))


{'인구': '331,002,651', '수도': 'Washington, D.C.', '주': '50', '국가': '미국'}

In [5]:
prompt = ''' 
인구, 수도, 주, 대한민국. JSON 객체로 반환
'''

completion = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{'role': 'user', 'content':prompt}],
    max_tokens=1024,
    temperature=0
)

print(completion)
json.loads(completion.choices[0].message.content)

ChatCompletion(id='chatcmpl-9pTgfiClZcHkzNrdKev91PLg1qyeu', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "인구": "51,780,579",\n  "수도": "서울",\n  "주": "대한민국"\n}', role='assistant', function_call=None, tool_calls=None))], created=1722055873, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=36, prompt_tokens=28, total_tokens=64))


{'인구': '51,780,579', '수도': '서울', '주': '대한민국'}

In [7]:
#첫 번째 쿼리에서는 인구가 정수(331002651)로 반환되고, 두 번째 쿼리에서는 인구가 문자열('32,730,000')로 반환
#첫 번째 응답은 주를 숫자(한 국가의 총 주 수)로 반환하고, 두 번째 응답은 주를 배열(주)로 반환
#즉, 응답의 불일치가 발생 문자열 

In [8]:
test_functions = [
    {
        'name': 'get_country_detrails',
        'description': 'Get the details of a country',
        'parameters': {
            'type': 'object',
            'properties': {
                'Population': {
                    'type': 'integer',
                    'description': '인구'
                },
                'Capital': {
                    'type': 'string',
                    'description': '수도'
                },
                'States': {
                    'type': 'string',
                    'description': '주'
                }
            }
        }
    }
]

In [10]:
import openai

country = '미국'
prompt = f'''
Population, Capital, States of {country}.
'''

completion = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role': 'user', 'content': prompt}],
    functions= test_functions,      # <======== add this parameter
    function_call='auto',           # <======== add this parameter
    max_tokens=1024,
    temperature=0 
)

print(completion)

ChatCompletion(id='chatcmpl-9pTrMhyXxYc0GsK0XzlRoF5mxB3Fx', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "Population": 331002651,\n  "Capital": "Washington, D.C.",\n  "States": "50"\n}', name='get_country_detrails'), tool_calls=None))], created=1722056536, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=37, prompt_tokens=81, total_tokens=118))


In [11]:
import json
print(json.loads(completion.choices[0].message.function_call.arguments))

{'Population': 331002651, 'Capital': 'Washington, D.C.', 'States': '50'}


In [12]:
import openai

country = '대한민국'
prompt = f''' 
Population, Capital, States of {country}.
'''

completion = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role': 'user', 'content':prompt}],
    functions=test_functions,       # <====== add this parameter
    function_call='auto',           # <====== add this parameter
    max_tokens=1024,
    temperature=0
)
print(completion)

ChatCompletion(id='chatcmpl-9pTui8MSdCWPZUnD9eBSFLgT0BSQZ', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "Population": 51780579,\n  "Capital": "서울",\n  "States": "경기도, 강원도, 충청북도, 충청남도, 전라북도, 전라남도, 경상북도, 경상남도, 제주도"\n}', name='get_country_detrails'), tool_calls=None))], created=1722056744, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=87, prompt_tokens=82, total_tokens=169))


In [13]:
import json
print(json.loads(completion.choices[0].message.function_call.arguments))

{'Population': 51780579, 'Capital': '서울', 'States': '경기도, 강원도, 충청북도, 충청남도, 전라북도, 전라남도, 경상북도, 경상남도, 제주도'}
